# Install

In [ ]:
!pip3 install pshmodule

In [ ]:
!pip3 install pickle5

In [ ]:
!pip3 install pandas==1.5.0

In [ ]:
!pip3 install openai

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data Load

In [6]:
import sys
sys.path.append('/content/drive/MyDrive/MemeProject/src/pytorch/augmentation')
print(sys.path)

['/content', '/env/python', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.10/dist-packages/IPython/extensions', '/root/.ipython', '/content/drive/MyDrive/MemeProject/src/pytorch/augmentation']


In [7]:
import re
import config as cfg
import pandas as pd
from tqdm import tqdm
from pshmodule.utils import filemanager as fm

In [8]:
df = fm.load(cfg.origin_class154_ref)

extension : .pickle
Loaded 181704 records from drive/MyDrive/MemeProject/data/augmentation/origin_class154_ref.pickle


In [9]:
# df = fm.load(cfg.userdict_aug)

In [10]:
df.head()

,num,g_num,u_num,material,speech,u,meme,arg1,arg_op1,arg_op2,label
0,1,1,1,학교생활,자랑,나 중간고사 반에서 1등했어,그렇다면 통 크게 함 쏴야지! 근데 이제 내 것도 곁들인...,중간고사,,,147
1,1,1,1,학교생활,자랑,나 중간고사 반에서 1등했어,그렇다면 통 크게 함 쏴야지! 근데 이제 내 몫을 제일 많이 곁들인...,중간고사,,,147
2,1,1,1,학교생활,자랑,나 중간고사 반에서 1등했어,그렇다면 통 크게 함 쏴야지! 근데 이제 대춘이 몫도 곁들인...,중간고사,,,147
3,1,1,1,학교생활,자랑,나 중간고사 반에서 1등했어,그렇다면 통 크게 함 쏴야지! 근데 이제 배보다 배꼽이 더 큰 걸 곁들인...,중간고사,,,147
4,1,1,1,학교생활,자랑,나 중간고사 반에서 1등했어,그렇다면 통 크게 함 쏴야지! 근데 이제 과소비를 곁들인...,중간고사,,,147


# Under Sample

In [11]:
df_temp = df[['u', 'material', 'speech', 'label']]

In [12]:
df_temp.drop_duplicates(inplace=True)
df_temp.reset_index(drop=True, inplace=True)

<ipython-input-12-3c628d64f46f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp.drop_duplicates(inplace=True)


In [13]:
df_temp.shape

(9141, 4)

100 미만 확인

In [14]:
count_dict = {(i[1]['material'], i[1]['speech'], i[1]['label']): 0 for i in df_temp.iterrows()}

In [15]:
for i in df_temp.iterrows():
  count_dict[(i[1]['material'], i[1]['speech'], i[1]['label'])] += 1

In [16]:
sorted_dict = dict(sorted(count_dict.items(), key = lambda item: item[1], reverse=True))

In [17]:
sorted_dict

{('밈봇', '명령', 151): 491,
 ('밈봇', '혐오', 153): 483,
 ('밈봇', '실망', 152): 480,
 ('밈봇', '만족', 150): 479,
 ('유명인', '칭찬', 89): 144,
 ('기후', '진술', 28): 144,
 ('패션', '칭찬', 143): 126,
 ('기념일', '계획', 18): 114,
 ('경사', '자랑', 9): 108,
 ('자기개발', '계획', 108): 102,
 ('전자기기', '칭찬', 119): 102,
 ('자기개발', '의지', 110): 102,
 ('음식', '칭찬', 101): 96,
 ('주거', '소망', 121): 96,
 ('인간관계', '칭찬', 107): 96,
 ('돈', '자랑', 33): 96,
 ('운동', '계획', 78): 95,
 ('문화예술', '칭찬', 41): 90,
 ('연애', '자랑', 75): 90,
 ('취미', '의지', 134): 90,
 ('음료', '의지', 92): 90,
 ('미용', '의지', 50): 90,
 ('직장생활', '의지', 128): 90,
 ('가정생활', '계획', 0): 90,
 ('미디어콘텐츠', '칭찬', 47): 84,
 ('교통', '소망', 13): 78,
 ('운동', '의지', 80): 78,
 ('반려동물', '칭찬', 59): 78,
 ('연애', '소망', 73): 78,
 ('돈', '소망', 31): 78,
 ('여행', '소망', 67): 77,
 ('반려동물', '자랑', 57): 77,
 ('음식', '진술', 100): 72,
 ('여행', '계획', 66): 72,
 ('학교생활', '진술', 148): 72,
 ('미디어콘텐츠', '진술', 46): 72,
 ('학교생활', '계획', 144): 72,
 ('직장생활', '소망', 127): 72,
 ('인간관계', '자랑', 105): 72,
 ('돈', '의지', 32): 66,
 ('쇼핑', '진술', 64): 

In [18]:
temp = {i[0][2]:i[1] for i in sorted_dict.items() if int(i[1]) < 30}

In [19]:
temp

{142: 24,
 22: 24,
 27: 24,
 109: 24,
 122: 24,
 111: 24,
 7: 24,
 116: 24,
 23: 24,
 99: 24,
 44: 24,
 88: 18,
 54: 18,
 45: 18,
 114: 18,
 112: 18,
 79: 18,
 93: 18,
 69: 18,
 11: 12,
 84: 12,
 149: 12,
 65: 12,
 131: 12,
 133: 12,
 56: 12,
 86: 12,
 24: 6,
 77: 6,
 30: 6,
 113: 6,
 35: 6,
 83: 6}

30미만 나누기

In [20]:
df_temp.shape

(9141, 4)

In [21]:
df_temp.head()

,u,material,speech,label
0,나 중간고사 반에서 1등했어,학교생활,자랑,147
1,중간고사 점수 내가 반에서 제일 잘 받음,학교생활,자랑,147
2,나 반에서 중간 성적 제일 좋아,학교생활,자랑,147
3,우리 반에서 내가 시험 제일 잘 봤다,학교생활,자랑,147
4,중간고사 반 1등 먹음,학교생활,자랑,147


In [22]:
# 30 이상
temp_upper = [[i[1]['material'], i[1]['speech'], i[1]['u'], i[1]['label']] for i in df_temp.iterrows() if i[1]['label'] not in temp]

In [23]:
df_temp_upper = pd.DataFrame(temp_upper, columns=['material', 'speech', 'u', 'label'])

In [24]:
df_temp_upper.shape

(8601, 4)

In [25]:
# 30 미만
temp_under = [[i[1]['material'], i[1]['speech'], i[1]['u'], i[1]['label']] for i in df_temp.iterrows() if i[1]['label'] in temp]

In [26]:
df_temp_under = pd.DataFrame(temp_under, columns=['material', 'speech', 'u', 'label'])

In [27]:
df_temp_under.shape

(540, 4)

In [28]:
df_temp_under.head()

,material,speech,u,label
0,경사,칭찬,결혼식 뷔페 개맛있었다,11
1,경사,칭찬,결혼식장 음식 JMT,11
2,경사,칭찬,결혼식 뷔페가 존나 쩔었어,11
3,경사,칭찬,결혼식 뷔페 레알 짱맛!,11
4,경사,칭찬,결혼식 식사가 오졌다니까,11


# chatGPT API

In [29]:
import openai

In [32]:
# 발급받은 API 키 설정
OPENAI_API_KEY = cfg.OPENAI_API_KEY

# openai API 키 인증
openai.api_key = OPENAI_API_KEY

# 모델 - GPT 3.5 Turbo 선택
# model = "gpt-3.5-turbo"
model = "gpt-4"

In [33]:
result = []

for i in tqdm(df_temp_under.iterrows()):
  # query = f"너의 역할은 이제 한국어 문장을 대화체의 말투로 변환하는 봇이야. '{i[1]['u']}'라는 문장을 한국어로 대화체의 말투로 변환해줘."
  query = f'"{i[1]["u"]}"라는 문장을 다른 반말 어투로 10개로 변환해줘'

  # 원본 입력
  result.append([i[1]['material'], i[1]['speech'], i[1]['u'], i[1]['label']])

  # 메시지 설정하기
  messages = [
          {"role": "system", "content": "You are a helper who extracts keywords."},
          {"role": "user", "content": query}
  ]

  # ChatGPT API 호출하기
  response = openai.ChatCompletion.create(
      model=model,
      messages=messages
  )
  answer = response['choices'][0]['message']['content']

  # 정규표현식 패턴: 숫자로 시작하며 마침표와 공백이 뒤따르는 패턴
  pattern = r'\d+[\.)]\s*'

  # re.sub 함수를 사용하여 패턴에 해당하는 부분을 제거합니다.
  answer_temp = re.sub(pattern, '', answer).split('\n')
  for a in answer_temp:
    # 정규 표현식으로 ""안에 문자 추출
    # m = re.search(r'(?<=").*?(?=")', a)
    if a:
      a = a.strip().replace('"','')
      result.append([i[1]['material'], i[1]['speech'], a, i[1]['label']])

2it [01:20, 40.28s/it]


KeyboardInterrupt: ignored

In [ ]:
df_temp_under_aug = pd.DataFrame(result, columns=['g_num', 'u_num', 'material', 'speech', 'u', 'arg1', 'arg_op1', 'arg_op2', 'user_dic', 'chatgpt', 'label'])

In [ ]:
df_temp_under_aug.head(10)

,g_num,u_num,material,speech,u,arg1,arg_op1,arg_op2,user_dic,chatgpt,label
0,740,1,돈,계획,다음 달에 크게 소비할 계획이야,,다음 달,,,False,30
1,740,1,돈,계획,다음 달에 돈 많이 쓸 거야.,,다음 달,,,True,30
2,740,1,돈,계획,다음 달에 큰 소비 계획이 있어.,,다음 달,,,True,30
3,740,1,돈,계획,내달에 많이 쓸 거야.,,다음 달,,,True,30
4,740,1,돈,계획,내달에 큰 소비할 거야.,,다음 달,,,True,30
5,740,1,돈,계획,다음 달 돈 많이 쓸 거야.,,다음 달,,,True,30
6,740,1,돈,계획,다음 달에 크게 소비할 계획이야,,,,,False,30
7,740,1,돈,계획,다음 달 큰 돈 쏴봐야겠어.,,,,,True,30
8,740,1,돈,계획,내달엔 쫌 쏴야지.,,,,,True,30
9,740,1,돈,계획,다음달엔 돈 더 써야겠다.,,,,,True,30


In [ ]:
df['chatgpt'] = 'False'

In [ ]:
df_temp = pd.concat([df, df_temp_under_aug])

In [ ]:
df_temp.shape

(122493, 11)

Save

In [ ]:
fm.save('drive/MyDrive/MemeProject/data/augmentation/chatgpt_aug.pickle', df_temp)

Saved 122493 records


Count Check

In [ ]:
count_dict = {(i[1]['material'], i[1]['speech'], i[1]['label']): 0 for i in df_temp.iterrows()}

In [ ]:
for i in df_temp.iterrows():
  count_dict[(i[1]['material'], i[1]['speech'], i[1]['label'])] += 1

In [ ]:
sorted_dict = dict(sorted(count_dict.items(), key = lambda item: item[1], reverse=True))

In [ ]:
sorted_dict

{('기념일', '계획', 18): 2508,
 ('학교생활', '자랑', 147): 2312,
 ('미용', '자랑', 51): 1929,
 ('자기개발', '의지', 110): 1867,
 ('반려동물', '자랑', 57): 1842,
 ('자기개발', '계획', 108): 1714,
 ('쇼핑', '칭찬', 65): 1673,
 ('인간관계', '진술', 106): 1646,
 ('학교생활', '소망', 145): 1637,
 ('반려동물', '진술', 58): 1591,
 ('교통', '진술', 16): 1571,
 ('미용', '소망', 49): 1515,
 ('기후', '진술', 28): 1429,
 ('여행', '계획', 66): 1285,
 ('자기개발', '자랑', 111): 1241,
 ('반려동물', '계획', 54): 1240,
 ('연애', '소망', 73): 1218,
 ('패션', '소망', 139): 1196,
 ('교통', '소망', 13): 1179,
 ('인간관계', '계획', 102): 1173,
 ('학교생활', '의지', 146): 1129,
 ('쇼핑', '진술', 64): 1122,
 ('음료', '진술', 94): 1113,
 ('학교생활', '계획', 144): 1100,
 ('직장생활', '의지', 128): 1090,
 ('연애', '의지', 74): 1071,
 ('미디어콘텐츠', '의지', 44): 1070,
 ('음식', '계획', 96): 1069,
 ('기후', '자랑', 27): 1053,
 ('미용', '칭찬', 53): 1045,
 ('음료', '자랑', 93): 1029,
 ('쇼핑', '자랑', 63): 1029,
 ('패션', '의지', 140): 1016,
 ('미디어콘텐츠', '자랑', 45): 1009,
 ('여행', '칭찬', 71): 993,
 ('전자기기', '칭찬', 119): 987,
 ('문화예술', '계획', 36): 983,
 ('연애', '칭찬', 77): 971,
 (